El archivo Vinos.csv está formado por las muestras de un análisis químico realizado a tres tipos distintos de vinos producidos en una misma región de Italia. El número de muestras considerado es el siguiente: 59 muestras del vino tipo 1, 71 del vino tipo 2 y 48 de del vino tipo 3. Cada muestra contiene, en primer lugar, el número del tipo al cual pertenece el vino analizado seguido por los 13 atributos que lo caracterizan.

      Por ejemplo, la siguiente fila: 2, 12.29, 3.17, 2.21, 18, 88, 2.85, 2.99, 0.45, 2.81, 2.3, 1.42, 2.83, 406

es el resultado del análisis de un vino correspondiente al tipo 2 (1er. valor de la fila) seguido por 13 valores que indican los niveles de las mediciones realizadas a dicho vino.

Si se utiliza una red con capa de salida softmax y 3 capas ocultas: la primera formada por 20 neuronas, la segunda formada por 10 neuronas y la tercera formada por 5 neuronas 

¿cuántos parámetros o pesos tiene la red?

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf

DATA_PATH="./Data/"

data=pd.read_csv(DATA_PATH+'Vinos.csv', sep=';')

X = data.drop('Class', axis=1)
T = data['Class']

scaler = preprocessing.StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

binarizer = preprocessing.LabelBinarizer()
T = binarizer.fit_transform(T)

# X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.2, random_state=42, stratify=T)


In [17]:
from gpuConfig import configure_cuda

configure_cuda()



TensorFlow está configurado para usar CUDA (GPU).


True

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

model = Sequential([
    Input(shape=(X.shape[1],)),
    Dense(20, activation='relu'),
    Dense(10, activation='relu'),
    Dense(5, activation='relu'),
    Dense(T.shape[1], activation='softmax')
])

display(model.summary())

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 20)             │           280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 5)              │            55 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 3)              │            18 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 563 (2.20 KB)

 Trainable params: 563 (2.20 KB)

 Non-trainable params: 0 (0.00 B)

None

A partir del archivo Vinos.csv, se entrenó un modelo de red neuronal multicapa (MLP) con una única capa oculta compuesta por 8 neuronas, con el objetivo de predecir el tipo de vino correspondiente a cada muestra. Todas las variables fueron normalizadas utilizando media y desvío estándar, y el conjunto completo de ejemplos se utilizó para el entrenamiento (sin división entre entrenamiento y prueba).

La función de activación de la capa de salida fue 'softmax' y se utilizó el optimizador SGD con una tasa de aprendizaje (learning_rate) de 0.05. El entrenamiento se realizó durante  50 épocas, con un tamaño de lote (batch size) de 20. Para garantizar la reproducibilidad de los resultados, se fijó la semilla aleatoria de TensorFlow con el siguiente comando:

import tensorflow as tf  

tf.random.set_seed(1)

Se evaluaron dos opciones para la función de activación de la capa oculta: 'sigmoid' y 'relu'. Luego del entrenamiento, se calculó el accuracy utilizando el mismo conjunto completo de datos como entrada.

Indique cuál de las siguientes afirmaciones es verdadera:

In [20]:
from tensorflow.keras.optimizers import SGD


tf.random.set_seed(1)
optimizer = SGD(learning_rate=0.05)

model = Sequential([
    Input(shape=(X.shape[1],)),
    Dense(8, activation='sigmoid'),
    Dense(T.shape[1], activation='softmax')
])

model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy', 
              metrics=['accuracy']
              )
history = model.fit(X, T, 
                    validation_data=(X, T),
                    epochs=50, 
                    batch_size=20,
                    verbose=0)

# Imprimir los accuracies almacenados en history (primer entrenamiento)
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# print("Accuracy por época (entrenamiento):")
# for i, a in enumerate(train_acc, start=1):
#   print(f"época {i}: {a:.4f}")

# print("\nAccuracy por época (validación):")
# for i, a in enumerate(val_acc, start=1):
#   print(f"época {i}: {a:.4f}")

print(f"\nAccuracy final (entrenamiento): {train_acc[-1]:.4f}")
print(f"Accuracy final (validación): {val_acc[-1]:.4f}")


tf.random.set_seed(1)
optimizer = SGD(learning_rate=0.05)

model = Sequential([
    Input(shape=(X.shape[1],)),
    Dense(8, activation='relu'),
    Dense(T.shape[1], activation='softmax')
])

model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy', 
              metrics=['accuracy']
              )
history = model.fit(X, T, 
                    validation_data=(X, T),
                    epochs=50, 
                    batch_size=20,
                    verbose=0)
# Imprimir los accuracies almacenados en history (primer entrenamiento)
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# print("Accuracy por época (entrenamiento):")
# for i, a in enumerate(train_acc, start=1):
#   print(f"época {i}: {a:.4f}")

# print("\nAccuracy por época (validación):")
# for i, a in enumerate(val_acc, start=1):
#   print(f"época {i}: {a:.4f}")

print(f"\nAccuracy final (entrenamiento): {train_acc[-1]:.4f}")
print(f"Accuracy final (validación): {val_acc[-1]:.4f}")



Accuracy final (entrenamiento): 0.9775
Accuracy final (validación): 0.9831

Accuracy final (entrenamiento): 0.9775
Accuracy final (validación): 0.9775


A partir de los ejemplos del archivo Vinos.csv se entrenó un multiperceptrón con una sola capa oculta formada por 10 neuronas para predecir el tipo de vino de una muestra dada. Las funciones de activación para las capas oculta y de salida son “tanh” y “sigmoid” respectivamente.

Indique cuáles de los siguientes factores inciden en la dirección de cambio (signo de la modificación) de los pesos de la red:

El valor de la derivada de la función de activación

El error cometido en la predicción